In [8]:
import os
import pandas as pd
import numpy as np
import json
from tqdm.auto import tqdm
from panda_challenge.models import ClassifcationDatasetMultiCropModel
from panda_challenge.dataset import ClassifcationMultiCropInferenceDataset
from torch.utils.data import DataLoader
from sklearn.metrics import cohen_kappa_score
import torch
import torch.nn as nn

In [9]:
with open('./configs/test_run_milticrop_config.json', 'r') as f:
    params = json.load(f)
data_val = pd.read_csv('/data/personal_folders/skolchenko/panda/data_val.csv')
data_val.head(5)

,image_id,data_provider,isup_grade,gleason_score,has_slide
0,90b8c7eac5e1df7eb3310d2346a24ebc,karolinska,0,0+0,True
1,ecae863e7c478594aa4c84ce132b3825,karolinska,1,3+3,True
2,d2fa7b7e8b5d7ac158a745839005336a,karolinska,2,3+4,True
3,e3c973ccb8b37ad37da0acd96be64830,karolinska,2,3+4,True
4,5c869a5a8bd2bd340fe13b188be333c1,karolinska,1,3+3,True


Load model and weights

In [10]:
model = ClassifcationDatasetMultiCropModel(
    params['model_name'],
    **params['model_config'])
model.cuda()
checkpoint = torch.load(os.path.join(params['log_dir'], 'checkpoints/best.pth'))
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

ClassifcationDatasetMultiCropModel(
  (enc): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

Define dataset and dataloader

In [11]:
dataset_val = ClassifcationMultiCropInferenceDataset(
        params['val_csv'],
        None,
        params['val_image_dir'],
        N = params['N'])
validation_loader = DataLoader(
    dataset_val,
    batch_size=params['batch_size'],
    num_workers=params['n_workers'],
    pin_memory=True,
    shuffle=False
    )

Run predictions, if necesarry, run tta

In [12]:
validation_preds = []
with torch.no_grad():
    for data_b in validation_loader:
        preds = model(data_b['features'].cuda())
        preds = nn.Softmax(dim=1)(preds)
        preds = preds.cpu().numpy()
        preds = np.argmax(preds, axis=1)
        validation_preds.extend(preds)

Compute metrics

In [13]:
cohen_kappa_score(
    validation_preds, 
    data_val.isup_grade.values, 
    weights='quadratic')

0.7421398975989665